<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sl_whisper_medium_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-s6_sa_fs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-s6_sa_fs
  Resolved https://github.com/huggingface/transformers to commit 238d2e3c44366aba9dc5c770c95475765a6725cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269614 sha256=36490bc2771a91d3bee244a0e8de00e55e1eb8faf14b0863424f5fa561e973de
  Stored in directory: /tmp/pip-ephem-wheel-cache-t6_d7gwx/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [57]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [58]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 309
})

In [59]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [60]:
model = whisper.load_model("medium")

In [61]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [62]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [63]:
text

,0
0,seveda lahko čestitamo za hiter razvoj elektro...
1,anžetu logarju in njegovim staršem je s smrtjo...
2,gre pa za to da imajo priložnost biti aktivni ...
3,zato je pomembno da evropska zveza kot sila mi...
4,vlado republike slovenije pozivam naj čim prej...
...,...
304,ta rešitev o kateri govorimo pa bo žal nezados...
305,sem za ukrepanje na področju podnebnih spremem...
306,sama ji nisem zelo naklonjena ker jo lahko raz...
307,beneška komisija je bila pri svojih zapisih ze...


In [64]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [65]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000
306,female,sama ji nisem zelo naklonjena ker jo lahko raz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00250244140625, -0.001983642578125, -0.002...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [66]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
297,male,nekateri seveda postavljajo tukaj dilemo ali n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.1724853515625, -0.188323974609375, -0.1896...",16000
302,male,sistem vračanja iz grčije v turčijo ne deluje ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00018310546875, 0.000946044921875, 9.155273...",16000
303,male,od komisije pričakujemo da bo predložila smise...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0035400390625, -0.00372314453125, -0.00039...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [67]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
7,female,na nekaj pa bi rada opozorila tukaj v svojem g...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, -0.003326416015625, -0.004821...",16000
12,female,iz tega razloga je tudi napačno sklepati da la...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.044647216796875, -0.03350830078125, -0.019...",16000
18,female,gre namreč za ugled nas vseh gre za ugled evro...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.03363037109375, -0.033935546875, -0.036132...",16000
20,female,z vsem spoštovanjem do novega organa in do izj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.1650390625, 0.13531494140625, 0.11520385742...",16000
26,female,v naši energetski mešanici je vse več premoga.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003143310546875, 0.00390625, 6.103515625e-0...",16000
...,...,...,...,...,...
300,female,moram reči da sem po dolgem času zadovoljna z ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004669189453125, 0.012481689453125, 0.00991...",16000
301,female,napačna je namreč razlaga da je kategorija ope...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.024261474609375, 0.028076171875, 0.03970336...",16000
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000


In [68]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sl", fp16=False)
    return result["text"]

In [69]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [70]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [71]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.4090909090909091, Precision: 0.7, Recall: 0.6363636363636364, F1: 0.6666666666666666
Processed 2 files, WER: 0.46153846153846156, Precision: 0.6153846153846154, Recall: 0.6153846153846154, F1: 0.6153846153846154
Processed 3 files, WER: 0.26666666666666666, Precision: 0.7333333333333333, Recall: 0.7333333333333333, F1: 0.7333333333333333
Processed 4 files, WER: 0.75, Precision: 0.55, Recall: 0.4782608695652174, F1: 0.5116279069767442
Processed 5 files, WER: 0.40625, Precision: 0.6071428571428571, Recall: 0.5862068965517241, F1: 0.5964912280701754
Processed 6 files, WER: 0.43902439024390244, Precision: 0.696969696969697, Recall: 0.6571428571428571, F1: 0.676470588235294
Processed 7 files, WER: 1.0, Precision: 0.4666666666666667, Recall: 0.4666666666666667, F1: 0.4666666666666667
Processed 8 files, WER: 0.6666666666666666, Precision: 0.6, Recall: 0.5294117647058824, F1: 0.5625
Processed 9 files, WER: 0.3076923076923077, Precision: 0.6666666666666666, Recall: 0.66

In [45]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.1, Precision: 0.23076923076923078, Recall: 0.3, F1: 0.2608695652173913
Processed 2 files, WER: 0.9090909090909091, Precision: 0.21052631578947367, Recall: 0.18181818181818182, F1: 0.1951219512195122
Processed 3 files, WER: 0.9090909090909091, Precision: 0.17391304347826086, Recall: 0.18181818181818182, F1: 0.17777777777777776
Processed 4 files, WER: 0.8095238095238095, Precision: 0.25, Recall: 0.23809523809523808, F1: 0.24390243902439024
Processed 5 files, WER: 0.7567567567567568, Precision: 0.28125, Recall: 0.2571428571428571, F1: 0.26865671641791045
Processed 6 files, WER: 1.0, Precision: 0.18181818181818182, Recall: 0.2, F1: 0.1904761904761905
Processed 7 files, WER: 0.8108108108108109, Precision: 0.1935483870967742, Recall: 0.1875, F1: 0.19047619047619047
Processed 8 files, WER: 0.8333333333333334, Precision: 0.19047619047619047, Recall: 0.17391304347826086, F1: 0.1818181818181818
Processed 9 files, WER: 0.75, Precision: 0.25, Recall: 0.25, F1: 0.25
Proces